Task1_0620. 네이버 영화 순위 사이트에서 영화제목, 가격, 타입(구매 or 대여) 정보를 가져와서 TITLE, PRICE, TYPE 3개의 컬럼과 100개의 데이터포인트로 구성된 데이터프레임을 출력하세요.

In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas as pd


url = 'https://serieson.naver.com/v3/movie/ranking/daily'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

sell_types_class = '.Price_text__pRk_f'
price_infos_class = '.Price_price__GqXqo'
names_class = '.Title_title__s9o0D'

title = soup.select(names_class)
title = [t.text for t in title]
price = soup.select(price_infos_class)
price = [p.text[:-2] for p in price]
type = soup.select(sell_types_class)
type = [n.text for n in type]

df = pd.DataFrame({'Title': title, 'Price': price, 'Type': type})
df

,Title,Price,Type
0,인사이드 아웃(패키지상품 : 더빙판 + 부가영상 추가증정),"7,150",구매
1,귀공자,"1,540",대여
2,설계자,"11,000",대여
3,명탐정 코난 VS 괴도 키드,"11,000",대여
4,파묘,"7,000",대여
...,...,...,...
95,시카고,"5,500",구매
96,해리 포터와 비밀의 방(2편),"1,300",대여
97,매트릭스,"1,300",대여
98,비포 선라이즈,"1,300",대여


In [9]:
#ex

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# URL 설정
url = 'https://serieson.naver.com/v3/movie/ranking/realtime'

# HTTP GET 요청
response = requests.get(url)
response.raise_for_status()  # 요청이 성공했는지 확인

# BeautifulSoup을 사용하여 HTML 파싱
soup = BeautifulSoup(response.content, 'html.parser')

# 영화 제목과 가격을 저장할 리스트
titles = []
prices = []
types = []

# 영화 제목과 가격 추출
movies = soup.select('.RankingList_ranking_list__N4QsH > li')

for movie in movies:
    title_tag = movie.select_one('.Title_title__s9o0D')
    if title_tag:
        title = title_tag.get_text(strip=True)
    
        # 가격 및 타입 추출
        price_items = movie.select('.Price_price_item__tOOqb')
        for price_item in price_items:
            type_tag = price_item.select_one('.Price_text__pRk_f')
            price_tag = price_item.select_one('.Price_price__GqXqo')
            if type_tag and price_tag:
                type_text = type_tag.get_text(strip=True)
                price = price_tag.get_text(strip=True)
                
                # 가격에서 "캐시" 및 SVG 태그 제거
                price = re.sub(r'캐시.*', '', price).strip()
                
                titles.append(title)
                prices.append(price)
                types.append(type_text)

# 데이터프레임 생성
df = pd.DataFrame({
    'Title': titles,
    'Price': prices,
    'Type': types
})

# 데이터프레임 출력
df


,Title,Price,Type
0,인사이드 아웃(패키지상품 : 더빙판 + 부가영상 추가증정),"7,150",구매
1,콩나물,"1,000",구매
2,태극기 휘날리며 - 특별판,"10,000",구매
3,설계자,"11,000",대여
4,원피스 필름 레드,"1,540",대여
...,...,...,...
81,마스크,"1,300",대여
82,사운드 오브 뮤직,"5,500",구매
83,마지막 한 걸음까지,500,구매
84,흑인 오르페,500,구매


Task2_0620. 앞에서 출력한 기사 리스트를 인덱스 번호와 텍스트 컬럼 2개로 구성된 pandas 데이터프레임으로 변환 후 csv 파일로 저장 후 다시 불러오세요.

In [5]:
import pandas as pd
import re

url = 'https://news.daum.net/politics#1'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

li = []
lines = soup.select('.link_txt')
for i, tag in enumerate(lines, 1):
    text = tag.get_text().strip().replace(',', '')
    matches = re.findall('[가-힣0-9]+', text)
    li.append((f"{i}. {' '.join(matches)}"))

num = []
title = []
for i in li:
    num.append(i.split('.')[0])
    title.append(i.split('.')[1])

df = pd.DataFrame({
    'Index': num,
    'Title': title
})
df
df.to_csv('article.csv', index=False)
df2 = pd.read_csv('article.csv')
df2

,Index,Title
0,1,조국 아무도 윤석열을 구해주지 못한다 맥베스와 아내의 최후 오고 있어
1,2,법사위 채상병 특검 입법 청문회 개최 박정훈 이종섭 출석 예정
2,3,경기교육행정포럼 학교 행정지원 제도 개선 모색
3,4,여야 국회의원 남해안권 발전 특별법 공동 발의
4,5,임병택 시흥시장 전시회 관람하며 환경 보호의 중요성 인식하길 바란다
5,6,임미애 영농형 태양광법 대표 발의
6,7,조태열 북러 조약 안보리 결의 명백 위반 유엔 무대서 규탄
7,8,경북도의회 순직 소방대원 지원 조례 발의
8,9,보훈차관 의료계 집단행동 속 보훈위탁 아산충무병원 진료체계 점검
9,10,포항 깜짝 방문 기회발전특구 8곳 발표


In [16]:
#ex
import re
import pandas as pd

soup = BeautifulSoup(response.text, 'html.parser')

lines = soup.select('body > div > main > section > div.main-sub > div > ul > li > strong > a')
results = []
for i, tag in enumerate(lines, start=1):
    text = tag.get_text().strip().replace(',', '')
    matches = re.findall('[가-힣0-9]+', text)
    result = ' '.join(matches)
    results.append({'Index':i, 'Text':result})
    
df = pd.DataFrame(results)
df
df.to_csv('news.csv', index=False)
# df = pd.read_csv('news.csv')
df


""


Task3_0620. url = 'https://news.daum.net/politics#1'은 정치기사 1~10페이지에 있는 기사를 모두 출력하세요. 

In [10]:
#task2
import requests
from bs4 import BeautifulSoup
import re

for j in range(1, 11):
    #https://news.daum.net/breakingnews/politics?page=1
    url = f'https://news.daum.net/breakingnews/politics?page={j}'
    print(url)
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    lines = soup.find_all('a',class_="link_txt")
    lines = lines[7:-21]
    for i, tag in enumerate(lines, 1):
        text = tag.get_text().strip().replace(',', '')
        matches = re.findall('[가-힣0-9]+', text)
        print(f"{i}. {' '.join(matches)}")
    print(f'페이지수: {j}')

https://news.daum.net/breakingnews/politics?page=1
1. 탈북단체 대북전단 30만장 살포
2. 북한 6 25 미제 반대 투쟁의 날 앞두고 원수 격멸 의지 굳건
3. 현대 폴란드 국제해양 안보포럼 서 잠수함 플랫폼 제안
4. 유승민 국민의힘 당대표 경선 불출마 절박함이 시작될 때 역할 하겠다 상보
5. 박찬대 건희위원회로 전락한 권익위 고무줄 청탁금지법 지적
6. 대통령실 야 단독소집 운영위 불참키로 야 증인 채택할듯
7. 속보 무의미한 도전이라 결론 유승민 당대표 불출마
8. 속보 유승민 전당대회 불출마 무의미한 도전
9. 원희룡 한동훈과 같은 날 같은 곳에서 당대표 출마 선언
10. 원희룡 23일 국회서 출마선언 과 같은 날 같은 장소 1시간 차이
11. 우크라 무기지원은 러 행동에 달려 155 포탄 등 방산 대기중
12. 박찬대 오늘 채해병 특검 청문회 윤 감추려는 진실 드러날 자리
13. 원희룡 모레 대표 출마선언 한동훈과 한날 출사표
14. 유승민 전대 불출마 무의미한 도전
15. 당 대표 출마 준비중인 한동훈 전 비대위원장 응원 화환
페이지수: 1
https://news.daum.net/breakingnews/politics?page=2
1. 당 대표 출마 준비중인 한동훈 전 비대위원장 응원 화환
2. 즐비하게 놓여져 있는 한동훈 응원 화환
3. 당 대표 출마 준비중인 한동훈 전 비대위원장 응원화환
4. 박찬대 해병대원 특검법 또다시 거부한다면 범죄자라 자백한 것으로 여길 것
5. 한동훈 전 비대위원장 사무실 앞 응원 화환
6. 유승민 당대표 불출마 무의미한 도전
7. 탈북민단체 어젯밤 파주서 대북전단 30만장 살포
8. 통합위 27일 2024 국민통합 컨퍼런스 개최
9. 속보 유승민 대표 경선 불출마 무의미한 도전이라 결론
10. 탈북민단체 어젯밤 대북 전단 30만 장 살포 북한 대응 주목
11. 유승민 당대표 불출마 선언 무의미한 도전
12. 자유북한운동연합 대북전단 살포
13. 북한 우크라전 끝나면 러시아에